In [1]:
import os
import shutil
import requests
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
!pwd

/projectnb/sparkgrp/ml-herbarium-grp/Spring2024/cvh_datasets


In [3]:
# input_dir should host all original datasets scraped from CVH
input_dir = "/projectnb/sparkgrp/ml-herbarium-grp/Spring2024/cvh_datasets"
# dataset_dir is the temporary csv file
dataset_dir = os.path.join(input_dir, "scraped_cvh_dataset.csv")
# cleaned_dataset_dir is the temporary csv file
cleaned_dataset_dir = os.path.join(input_dir, "cleaned_cvh_dataset.csv")
# sampled_dataset_dir is the sampled csv file
sampled_dataset_dir = os.path.join(input_dir, "sampled_cvh_dataset.csv")
# images_dir is the path for all images
images_dir = os.path.join(input_dir, "images")
# samples_dir is the path for samples used for validation
samples_dir = os.path.join(input_dir, "samples")

In [4]:
# Examine dataframe
df = pd.read_csv(dataset_dir)
df.head()

,Collection ID,Image Link,Phylum (门),Order (目),Family (科),Genus (属),Scientific Name,Chinese Name,Identified By,Date Identified,Collector,Collection Number,Collection Date,Collection Location,Altitude,Habitat,Phenology
0,e953d062,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,大麻科,山黄麻属,Trema nitida C.J.Chen,银毛叶山黄麻,林祁,20100702.0,NaN,960097,19960605.0,中国 四川省,1025.0,NaN,有花无果
1,e953c8c7,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,大麻科,山黄麻属,Trema levigata Hand.-Mazz.,羽脉山黄麻(麻椰树),林祁,20100702.0,Wuyi Exp.,00279,19790814.0,中国 福建省,700.0,河边灌丛中,有花有果
2,e953cf4c,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,大麻科,山黄麻属,Trema orientalis Blume,异色山黄麻,林祁,20100702.0,陈家瑞,93016,19930918.0,中国 广东省,NaN,林缘,有花无果
3,e953ca69,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,大麻科,山黄麻属,Trema levigata Hand.-Mazz.,羽脉山黄麻(麻椰树),林祁,20100702.0,任再舍,1-3712,19930515.0,中国 四川省,1050.0,路边,有花无果
4,e953cc08,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,大麻科,山黄麻属,Trema levigata Hand.-Mazz.,羽脉山黄麻(麻椰树),林祁,20100702.0,金效华,515,20020921.0,中国 四川省,1030.0,沟边,有花无果


In [5]:
# During the initial web scraping phrase, only rows where all elements were unable to retried
# were pruned, e.g. the photo link, taxons, .... Now we want to prune further. Approximately 
# half will be dropped here.
df.replace('missing', pd.NA, inplace=True)
# Drop rows where all elements are missing (NaN)
df.dropna(how='any', inplace=True)

df.nunique()

Collection ID          15034
Image Link             14689
Phylum (门)                 1
Order (目)                 62
Family (科)               176
Genus (属)                823
Scientific Name         3041
Chinese Name            3039
Identified By            371
Date Identified         1441
Collector               1276
Collection Number       9733
Collection Date         4455
Collection Location       38
Altitude                1760
Habitat                 4404
Phenology                  6
dtype: int64

In [34]:
# Saving the filtered dataset to a csv file, await for inputting to OCR
# REMEMBER TO UTF-8-SIG since there are many Chinese characters.
df.to_csv(cleaned_dataset_dir, index=False, encoding="utf-8-sig")

In [14]:
def download_image(url, file_path):
    """Download an image from `url` to `file_path`."""
    try:
        response = requests.get(url, timeout=10)  # Timeout to avoid hanging on slow responses
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            # return f"Downloaded: {file_path}"
        else:
            return f"Failed to download {url} - Status Code: {response.status_code}"
    except requests.RequestException as e:
        return f"Failed to download {url} - Exception: {str(e)}"

def download_images(data, folder, num_workers=10):
    # Create the directory if it does not exist
    if not os.path.exists(folder):
        os.makedirs(folder)

    tasks = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Iterate over the image links in the dataframe and prepare download tasks
        for index, row in data.iterrows():
            image_url = row['Image Link']
            collection_id = row['Collection ID']
            file_extension = '.jpg' if 'jpg' in image_url.lower() else '.png'
            file_name = f"{collection_id}{file_extension}"
            full_file_path = os.path.join(folder, file_name)
            # Schedule the download task
            tasks.append(executor.submit(download_image, image_url, full_file_path))

        # Progress bar monitoring
        for task in tqdm(as_completed(tasks), total=len(tasks), desc="Downloading Images"):
            print(task.result())

# Call the function to download images
download_images(df, images_dir, num_workers=10)

None
None


None
None
None


None


None


None


...

None


None
None
None


None


None
None


None


In [6]:
def count_unique_images(directory):
    # Set to store unique filenames
    unique_files = set()

    # Supported file extensions
    extensions = ('.jpg', '.jpeg', '.png')

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file ends with a supported extension
            if file.lower().endswith(extensions):
                # Add the file to the set of unique files
                unique_files.add(file)

    # Return the count of unique files
    return len(unique_files)

# Directory to check
directory_path = images_dir

# Get the count of unique image files
count = count_unique_images(directory_path)
print(f"Number of unique image files: {count}")


Number of unique image files: 15039


# Image Link Reptition
It appears that some time CVH dataset host empty image with this:

https://www.cvh.ac.cn/public/images/no_image.jpg

For example:

https://www.cvh.ac.cn/spms/detail.php?id=e929cc81

In [7]:
no_img_link = "https://www.cvh.ac.cn/public/images/no_image.jpg"

df_filtered = df.copy(deep=True)
df_filtered = df_filtered.drop_duplicates(subset='Collection ID', keep='first')
df_filtered = df_filtered[df_filtered['Image Link'] != no_img_link]

In [8]:
np.random.seed(42)
df_sampled = df_filtered.sample(n=1000)

In [9]:
df_sampled.nunique()

Collection ID          1000
Image Link             1000
Phylum (门)                1
Order (目)                44
Family (科)              106
Genus (属)               307
Scientific Name         683
Chinese Name            683
Identified By           141
Date Identified         411
Collector               334
Collection Number       950
Collection Date         683
Collection Location      33
Altitude                442
Habitat                 592
Phenology                 6
dtype: int64

In [10]:
df_sampled.head()

,Collection ID,Image Link,Phylum (门),Order (目),Family (科),Genus (属),Scientific Name,Chinese Name,Identified By,Date Identified,Collector,Collection Number,Collection Date,Collection Location,Altitude,Habitat,Phenology
16439,e922cfb7,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,水龙骨目,碗蕨科,鳞盖蕨属,Microlepia hookeriana (Wall. ex Hook.) C.Presl,虎克鳞盖蕨,Wei Ran,20100104.0,蒋日红等,016,20090916.0,中国 广西壮族自治区,460,山坡，疏林，瘠土,有孢子囊
30910,e953de5d,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,荨麻科,冷水花属,Pilea angulata Blume subsp. petiolaris (Sieb.e...,长柄冷水花,林祁,20100702.0,刘正宇,1802,19940626.0,中国 重庆市,740,溪边,有花无果
13185,e92cc55c,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,毛茛目,毛茛科,乌头属,Aconitum pilopetalum W.T.Wang & L.Q.Li,毛瓣乌头,罗艳,200108.0,罗艳,355,20010829.0,中国 四川省,3300~3700,路旁杂草丛中,有花无果
30499,e954cad7,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,蔷薇目,荨麻科,冷水花属,Pilea cavaleriei Leveille,波缘冷水花,林祁,20100524.0,李光照,13580,19931202.0,中国 广西壮族自治区,140,石灰岩 山坡,有花无果
10896,e9339d52,https://www.cvh.ac.cn/controller/spms/image.ph...,维管植物门,壳斗目,壳斗科,柯属,Lithocarpus iteaphyllus (Hance) Rehder,鼠刺叶柯,李安仁,20101100.0,青藏队,11957,19830710.0,中国 四川省,2400.0,常绿阔叶林中,无花有果


In [74]:
sampled_df.to_csv(sampled_dataset_dir, index=False, encoding="utf-8-sig")

In [71]:
# Iterate over the rows in sampled_df
for _, row in sampled_df.iterrows():
    # Construct the file name as 'Collection ID'.jpg
    file_name = str(row['Collection ID']) + '.png'
    source_file = os.path.join(source_dir, file_name)
    destination_file = os.path.join(samples_dir, file_name)
    
    # Check if the file exists in the source directory
    if os.path.exists(source_file):
        # Copy the file to the 'samples' directory
        shutil.copy(source_file, destination_file)
    else:
        print(f"File {file_name} does not exist in the source directory.")

print("Files copied successfully.")

Files copied successfully.


# Validation in ten batches

In [20]:
# Define the base directory for samples and ensure it exists
base_samples_dir = './batches_samples'
os.makedirs(base_samples_dir, exist_ok=True)

# Source directory where the original images are stored
source_dir = './samples/'

# Function to copy files
def copy_and_zip_files(df, batch_name):
    # Directory to store specific batch samples
    batch_dir = os.path.join(base_samples_dir, batch_name)
    os.makedirs(batch_dir, exist_ok=True)
    
    # Copy files
    for _, row in df.iterrows():
        file_name = str(row['Collection ID']) + '.png'
        source_file = os.path.join(source_dir, file_name)
        destination_file = os.path.join(batch_dir, file_name)
        if os.path.exists(source_file):
            shutil.copy(source_file, destination_file)
        else:
            print(f"File {file_name} does not exist in the source directory.")
    
    # Zip the directory
    shutil.make_archive(os.path.join(base_samples_dir, batch_name), 'zip', batch_dir)
    
    # Save DataFrame to CSV
    df.to_csv(os.path.join(base_samples_dir, f"{batch_name}.csv"), index=False)

# Process in batches of 100
for i in range(0, len(df_sampled), 100):
    batch_df = df_sampled.iloc[i:i+100]
    batch_number = i // 100 + 1
    batch_name = f'batch_{batch_number}_100'
    copy_and_zip_files(batch_df, batch_name)

print("Processing and zipping of batches complete.")


Processing and zipping of batches complete.


In [21]:
first_100.to_csv('./first_100_samples.csv', index=False)
next_900.to_csv('./next_900_samples.csv', index=False)